In [40]:
from __future__ import division

# Compute True, False positive and False negative for each tag

In [17]:
train_tags = {}
all_tags = set()
with open("./rawInput/train.csv", "r") as train_labels:
    train_labels.readline()
    for line in train_labels:
        img, tags = line.rstrip("\n").split(",")
        train_tags[img] = tags.split(" ")
        all_tags.update(train_tags[img])
print len(train_tags)

40479


In [18]:
pred_tags = {}
with open("./predict/train-predict.csv", "r") as predict_labels:
    predict_labels.readline()
    for line in predict_labels:
        img, tags = line.rstrip("\n").split(",")
        pred_tags[img] = tags.split(" ")
print len(pred_tags)

40479


In [49]:
# First value is number of correctly predicted
# Second is number of not detected (false negative)
# Third is number of inaccurately predicted (false positive)
tags_score = {t: [0,0,0] for t in all_tags}

for img in train_tags:
    true_tags = train_tags[img]
    predicted_tags = pred_tags[img]
    for t in true_tags:
        if t in predicted_tags:
            tags_score[t][0] += 1
        else:
            tags_score[t][1] += 1
    for t in predicted_tags:
        if t not in true_tags:
            tags_score[t][2] += 1

for t in tags_score:
    print t, tags_score[t]
    print "Precision", tags_score[t][0] / (tags_score[t][0] + tags_score[t][2])
    print "Recall", tags_score[t][0] / (tags_score[t][0] + tags_score[t][1])

selective_logging [337, 3, 5]
Precision 0.985380116959
Recall 0.991176470588
cultivation [4481, 66, 13]
Precision 0.997107254117
Recall 0.985484935122
clear [28177, 26, 119]
Precision 0.995794458581
Recall 0.999078112258
habitation [3646, 16, 26]
Precision 0.992919389978
Recall 0.99563080284
conventional_mine [100, 0, 7]
Precision 0.934579439252
Recall 1.0
cloudy [2218, 112, 40]
Precision 0.982285208149
Recall 0.951931330472
primary [37818, 22, 105]
Precision 0.997231231706
Recall 0.999418604651
water [7245, 17, 71]
Precision 0.990295243302
Recall 0.997659047094
haze [2663, 32, 19]
Precision 0.992915734526
Recall 0.988126159555
slash_burn [208, 1, 4]
Precision 0.981132075472
Recall 0.995215311005
partly_cloudy [7239, 12, 4]
Precision 0.999447742648
Recall 0.998345055854
artisinal_mine [334, 5, 0]
Precision 1.0
Recall 0.985250737463
blooming [330, 2, 6]
Precision 0.982142857143
Recall 0.993975903614
bare_ground [842, 17, 11]
Precision 0.987104337632
Recall 0.980209545984
blow_down [98, 

# Compute F2 for training set

In [28]:
LABELS = ['clear', 'cloudy', 'haze', 'partly_cloudy', 'agriculture', 'artisinal_mine', 'bare_ground', 'blooming', 'blow_down', 'conventional_mine', 'cultivation', 'habitation', 'primary', 'road', 'selective_logging', 'slash_burn', 'water']

In [47]:
import numpy as np
from sklearn.metrics import fbeta_score
def F2Score(predicted, actual):
    # see https://www.kaggle.com/c/planet-understanding-the-amazon-from-space#evaluation
    predicted = set(predicted)
    actual = set(actual)
    tp = len(predicted & actual)
    tn = len(LABELS) - len(predicted | actual)
    fp = len(predicted) - tp
    fn = (len(LABELS) - len(predicted)) - tn
    p = tp / (tp + fp)
    r = tp / (tp + fn)
    if p == 0 or r == 0:
        return 0
    b = 2
    return (1 + b**2) * p * r / (b**2*p + r)

In [48]:
sum_f2 = 0
num_imgs = 0
for img in train_tags:
    true_tags = train_tags[img]
    predicted_tags = pred_tags[img]
    sum_f2 += F2Score(true_tags, predicted_tags)
    num_imgs += 1
print sum_f2 / num_imgs

0.995134762365
